In [2]:
import os
os.getcwd()

'/home/stankvla/Projects/Python/DiplomaThesis/notebooks'

In [3]:
os.getcwd()

'/lnet/express/work/people/stankov/alignment/Thesis/notebooks'

# Padding
Padding when `center=True`

In [1]:
import torch
import torch.nn.functional as F

In [14]:
n_fft = 4

input = torch.arange(1, 10).unsqueeze(0) + 0.
(input, input.shape)

(tensor([[1., 2., 3., 4., 5., 6., 7., 8., 9.]]), torch.Size([1, 9]))

In [15]:
signal_dim = input.dim()
signal_dim

2

In [16]:
extended_shape = [1] * (3 - signal_dim) + list(input.size())
extended_shape

[1, 1, 9]

In [18]:
pad = int(n_fft // 2)
padded = F.pad(input.view(extended_shape), [pad, pad], 'reflect')
(padded, padded.shape)

(tensor([[[3., 2., 1., 2., 3., 4., 5., 6., 7., 8., 9., 8., 7.]]]),
 torch.Size([1, 1, 13]))

In [21]:
padded.shape[-signal_dim:]

torch.Size([1, 13])

In [20]:
padded.view(padded.shape[-signal_dim:])

tensor([[3., 2., 1., 2., 3., 4., 5., 6., 7., 8., 9., 8., 7.]])

In [25]:
input.size(-1)

9

In [24]:
padded.size(-1)


13

Calculate number of frames after stft with padding

`n_frames = (len(wav) - window_size) / hop_size + 1`

Suppose: 
* `n_fft` is even,
* `window_size = n_fft`,
* `hop_size = window_size / 2 = n_fft / 2`
* we applied padding `len(wav_padded) = len(wav_orig) + n_fft`


Then
`n_frames = (len(wav_padded) - window_size) / hop_size + 1` 

Thus
`n_frames = len(wav_orig) / hop_size + 1 = 2len(wav_orig) / n_fft + 1`

In [ ]:
def compute_frames(wave, n_fft):
    return 2 * wave.size(-1) // n_fft + 1

------------

In [38]:
a = torch.arange(60).reshape(3, 2, 10)
a

tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]],

        [[20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]],

        [[40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
         [50, 51, 52, 53, 54, 55, 56, 57, 58, 59]]])

In [39]:
b = torch.arange(0, -60, -1).reshape(3,2,10)
b

tensor([[[  0,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9],
         [-10, -11, -12, -13, -14, -15, -16, -17, -18, -19]],

        [[-20, -21, -22, -23, -24, -25, -26, -27, -28, -29],
         [-30, -31, -32, -33, -34, -35, -36, -37, -38, -39]],

        [[-40, -41, -42, -43, -44, -45, -46, -47, -48, -49],
         [-50, -51, -52, -53, -54, -55, -56, -57, -58, -59]]])

In [41]:
c = torch.arange(0, 600, 10).reshape(3,2,10)
c

tensor([[[  0,  10,  20,  30,  40,  50,  60,  70,  80,  90],
         [100, 110, 120, 130, 140, 150, 160, 170, 180, 190]],

        [[200, 210, 220, 230, 240, 250, 260, 270, 280, 290],
         [300, 310, 320, 330, 340, 350, 360, 370, 380, 390]],

        [[400, 410, 420, 430, 440, 450, 460, 470, 480, 490],
         [500, 510, 520, 530, 540, 550, 560, 570, 580, 590]]])

In [43]:
torch.cat([a,b, c], axis=1)

tensor([[[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9],
         [ 10,  11,  12,  13,  14,  15,  16,  17,  18,  19],
         [  0,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9],
         [-10, -11, -12, -13, -14, -15, -16, -17, -18, -19],
         [  0,  10,  20,  30,  40,  50,  60,  70,  80,  90],
         [100, 110, 120, 130, 140, 150, 160, 170, 180, 190]],

        [[ 20,  21,  22,  23,  24,  25,  26,  27,  28,  29],
         [ 30,  31,  32,  33,  34,  35,  36,  37,  38,  39],
         [-20, -21, -22, -23, -24, -25, -26, -27, -28, -29],
         [-30, -31, -32, -33, -34, -35, -36, -37, -38, -39],
         [200, 210, 220, 230, 240, 250, 260, 270, 280, 290],
         [300, 310, 320, 330, 340, 350, 360, 370, 380, 390]],

        [[ 40,  41,  42,  43,  44,  45,  46,  47,  48,  49],
         [ 50,  51,  52,  53,  54,  55,  56,  57,  58,  59],
         [-40, -41, -42, -43, -44, -45, -46, -47, -48, -49],
         [-50, -51, -52, -53, -54, -55, -56, -57, -58, -59],
         [400, 410, 

In [50]:
result = torch.cat([a,b, c], dim=1).permute(0, 2, 1)
result

tensor([[[  0,  10,   0, -10,   0, 100],
         [  1,  11,  -1, -11,  10, 110],
         [  2,  12,  -2, -12,  20, 120],
         [  3,  13,  -3, -13,  30, 130],
         [  4,  14,  -4, -14,  40, 140],
         [  5,  15,  -5, -15,  50, 150],
         [  6,  16,  -6, -16,  60, 160],
         [  7,  17,  -7, -17,  70, 170],
         [  8,  18,  -8, -18,  80, 180],
         [  9,  19,  -9, -19,  90, 190]],

        [[ 20,  30, -20, -30, 200, 300],
         [ 21,  31, -21, -31, 210, 310],
         [ 22,  32, -22, -32, 220, 320],
         [ 23,  33, -23, -33, 230, 330],
         [ 24,  34, -24, -34, 240, 340],
         [ 25,  35, -25, -35, 250, 350],
         [ 26,  36, -26, -36, 260, 360],
         [ 27,  37, -27, -37, 270, 370],
         [ 28,  38, -28, -38, 280, 380],
         [ 29,  39, -29, -39, 290, 390]],

        [[ 40,  50, -40, -50, 400, 500],
         [ 41,  51, -41, -51, 410, 510],
         [ 42,  52, -42, -52, 420, 520],
         [ 43,  53, -43, -53, 430, 530],
         [ 4

In [48]:
x = result[0]
x

tensor([[  0,  10,   0, -10,   0, 100],
        [  1,  11,  -1, -11,  10, 110],
        [  2,  12,  -2, -12,  20, 120],
        [  3,  13,  -3, -13,  30, 130],
        [  4,  14,  -4, -14,  40, 140],
        [  5,  15,  -5, -15,  50, 150],
        [  6,  16,  -6, -16,  60, 160],
        [  7,  17,  -7, -17,  70, 170],
        [  8,  18,  -8, -18,  80, 180],
        [  9,  19,  -9, -19,  90, 190]])

In [49]:
x[:4]

tensor([[  0,  10,   0, -10,   0, 100],
        [  1,  11,  -1, -11,  10, 110],
        [  2,  12,  -2, -12,  20, 120],
        [  3,  13,  -3, -13,  30, 130]])